## Imports

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import gc
import random
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import pytorch_lightning as pl
from archive import CustomGraphMultipleChoiceEncoder, CustomGraphMLMEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, GroupKFold
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Batch, Data
from scipy.stats import rankdata
from tqdm.auto import tqdm, trange
from sklearn.metrics import accuracy_score
from transformers import (
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    AutoModel,
    AutoTokenizer,
    AutoModelForMultipleChoice,
    AutoConfig,
    DataCollatorForLanguageModeling
)
import wandb
import random
pl.seed_everything(56) # для воспроизводимости экспериментов задаём random_seed 
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Seed set to 56


## Config of experiment

In [4]:
class CFG:
    class data:
        train_path = 'csqa_data/train.statement.jsonl' #путь к тренировочным данным для csqa
        dev_path = 'csqa_data/dev.statement.jsonl' #путь к данным для валидации csqa
        test_path = 'csqa_data/test.statement.jsonl' #путь к тестовым данным для csqa
        dream_train_path = 'data/train.statement.jsonl' #путь к тренировочным данным для dream
        dream_dev_path = 'data/dev.statement.jsonl' #путь к данным для валидации dream
        dream_test_path = 'data/test.statement.jsonl' #путь к тестовым данным для dream
        dream_train_concept_path = 'prep_data/train_choices.csv' #вершины графа commonsense соответствующие вариантам ответов для dream для train данных
        dream_dev_concept_path = 'prep_data/vall_choices.csv'  #вершины графа commonsense соответствующие вариантам ответов для dream для dev данных
        dream_test_concept_path = 'prep_data/test_choices.csv' #вершины графа commonsense соответствующие вариантам ответов для dream для test данных
        tokenizer = "microsoft/deberta-v3-large" # токенайзер, выбранной нами языковой модели
        graph_path = './graph.pkl' # словарь, состоящий из самого CommonSense графа в формате nx.DiGraph и его мета данных
        embeds_path = 'deberta_large_embedsV2.npy' # векторные представления вершин для графа 
        maper_path = 'csqa_data/maper_inv.pkl' # словарь, который сопоставляет индексу вершины графа саму вершину 
        concepts_lst = 'maper_v2.pkl' # словарь, который вершине графа сопоставляет индекс 
        dop_concepts_csqa_path = 'csqa_data/all_candsV3.pkl' # вершины графа, полученные из текста с помощью EntityLinking и BGE модели для csqa
        dop_concepts_dream_path = './dream_all_candidates.pkl' # вершины графа, полученные из текста с помощью EntityLinking и BGE модели для dream
        num_workers = 0 # количество cpu для мультипроцессинга
        pad_to_nodes = 64 # максимальное количество вершин
        batch_size = 4  # размер батча
        use_prefix = True # использование префикса для текста - Question: ... Answer ...
        max_length = 90  # максимальное число токенов в тексте
        seed = 56
    class model:
        model = "microsoft/deberta-v3-large" # выбранная нами языковая модель 
        optim = torch.optim.AdamW # оптимайзер
        use_only_encoder = False # Данный параметр используется для изъятия из модели только encoder части (требовался нам для экспериментов с T5)
        use_tkds = True # нужно ли использовать token_type_ids 
        grad_acum_steps = 1 # gradiet accumulation steps 
        not_padding_token = False # есть ли необходимость добавлять в модель padding_token
        torch_dtype = None  # тип весов модели (None -> fp32)
        scheduler = 'cosine' # вид используемого lr scheduler 
        warmup_steps = 0.0 # какая доля шагов модели будет приходится на warmup steps
        num_labels = 1 # Размер выхода модели
        label_smoothing = 0.0 # label smoothing - используется для регуляризации
        lr = 2e-6 # learing rate, используемый для текстового encoder модели
        lr_gnn = 1e-5 # learing rate, используемый для gnn части модели
        lr_fn = 1e-4 # learing rate, используемый для финального слоя модели
        cls_drop_type = None # вид DropOut [StableDropOut, MultiSampleDropout,  SimpleDropout==nn.Dropout]
        cls_drop = 0.0 # p of Dropout
        pool = 'attention' # вид пулинга используемый в модели
        max_epoches = 5  # максимальное количетсво эпох                   
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"] # Слои для которых будет отключён weights decay
        turn_off_drop = True # Выключение внутреннего dropout у encoder модели (attention_probs_dropout_prob, hidden_)
        num_cycles = 0.5 # параметр используемый в cosine_schedule_with_warmup
        eps = 1e-7 # epsilon в оптимайзере
        weight_decay = 0.0 # weight_decay encoder и gnn части модели
        weight_decay_fn = 0.0 # weight_decay головы модели
        betas = (0.9, 0.999)  # betas оптимайзера
        use_lora = False  # использовать ли lora адаптер при обучении
        class gnn:
            pool_type = 'no' # тип пулинга в gnn (no так как пулинг нам не нужен)
            num_nodes = 64 # кол-во вершин, подающихся в gnn
            max_nodes = 64 # кол-во вершин, подающихся в encoder (max_nodes <= num_nodes)
            edge_dim = 17 # кол-во различных связей между вершинами
            blocks = [  # параметры каждого графового слоя ("block_type" - тип слоя) & ('block_norm' - слой нормализация) & ('drop_p' - node dropout) & ('input_dim' - входная размерность)  & ('out_dim' - выходная размерность) & ('act' - слой активации)
                {'block_type':'gatv2','act':nn.GELU(),'block_norm':None,'drop_p':0.0,'input_dim':1024,'out_dim':1024,'heads':2},
                {'block_type':'gatv2','act':nn.GELU(),'block_norm':None,'drop_p':0.0,'input_dim':2048,'out_dim':1024,'heads':1},
                {'block_type':'mlp','act':nn.Identity(),'block_norm':None,'drop_p':0.0,'input_dim':1024,'out_dim':1024},
            ]

    seed = 56
    fold_number = 0

def set_wandb_cfg():
    """Функция, которая преобразует CFG в тип, пригодный для wandb"""
    config = {}
    for k,v in CFG.model.__dict__.items():
        if '__' not in k:
            config[k] = v
    for k,v in CFG.data.__dict__.items():
        if '__' not in k:
            config[k] = v
    config['fold_number'] = CFG.fold_number
    return config

## Data Utils

### Предобработка данных

In [5]:
def make_df_csqa(path,is_test=False):
    """Функция предобрабатывающая датасет csqa"""
    data = pd.read_json(path,lines=True)
    df = pd.DataFrame()
    
    df['question_state'] = data['question'].apply(lambda x: x['question_concept'])
    df['question'] = data['question'].apply(lambda x: x['stem'])
    df['choice_A'] = data['question'].apply(lambda x: x['choices'][0]['text'])
    df['choice_B'] = data['question'].apply(lambda x: x['choices'][1]['text'])
    df['choice_C'] = data['question'].apply(lambda x: x['choices'][2]['text'])
    df['choice_D'] = data['question'].apply(lambda x: x['choices'][3]['text'])
    df['choice_E'] = data['question'].apply(lambda x: x['choices'][4]['text'])
    
    if not is_test:
        df['label'] = data['answerKey'].map({'A':0,'B':1,'C':2,'D':3,'E':4})
    else:
        df['label'] = 0
    return df

In [6]:
def make_df_dream(path,concept_path, is_test=False):
    """Функция предобрабатывающая датасет dream"""
    data = pd.read_json(path,lines=True)
    df = []
    
    for i in range(len(data)):
        row = data.iloc[i]
        for ques in row[1]:
            df.append({
                'dialog_id': i,
                'dialog': row[0],
                'question': ques['question'],
                'choice_0': ques['choice'][0],
                'choice_1': ques['choice'][1],
                'choice_2': ques['choice'][2],
                'labels': -100 if is_test else ques['choice'].index(ques['answer'])
            })
    return pd.DataFrame(df).join(pd.read_csv(concept_path)).drop(columns=['choice_0', 'choice_1', 'choice_2']).rename(columns={'choice_1_concept': 'choice_0', 'choice_2_concept': 'choice_1', 'choice_3_concept': 'choice_2'})

### Датасеты

In [7]:
class GraphProcessorV2():
    def __init__(self, G, embeds, n_hop=1,pad_to_nodes=64):
        self.G = G
        self.embeds = embeds
        self.pad_to_nodes = pad_to_nodes
        self.n_hop = n_hop
        self.masking_p = 0.0
        self.labels = [] #cache labels like a atribute
        
    def get_k_hop_neighbours(self,node):
        if self.n_hop == 1:
            return list(self.G.neighbors(node))
        nbrs = set([node])
        for l in range(self.n_hop):
            nbrs |= set((nbr for n in nbrs for nbr in self.G.neighbors(n)))
        return list(nbrs)
            
    def process(self,nodes):
        if len(nodes) >= self.pad_to_nodes:
            nodes = nodes[:self.pad_to_nodes]
        else:
            for n in nodes:
                nodes += self.get_k_hop_neighbours(n)
                if len(nodes) >= self.pad_to_nodes:
                    break
        if len(nodes) >= self.pad_to_nodes:
            nodes = nodes[:self.pad_to_nodes]
        else:
            nodes += [0] * (self.pad_to_nodes - len(nodes))
            
        if self.masking_p > 0.0:
            prob_mat = [random.random() < self.masking_p for _ in nodes]
            prob_mat[0] = False # answer node
            nodes = [(x if f else 160892) for f,x in zip(prob_mat,nodes)] #160892 - padding_token
            labels = [(-100 if f else x) for f,x in zip(prob_mat,nodes)]
            self.labels.append(labels)
            
        inv_nodes = {n:i for i,n in enumerate(nodes)}    
        sub_graph = self.G.subgraph(nodes)
        edge_attrs,edge_index = [], []
        for node1,node2,edge_feat in sub_graph.edges(data='relation'):
            edge_index.append([inv_nodes[node1], inv_nodes[node2]])
            edge_attrs.append(edge_feat)
        nodes = [n if n < 799272 else 160892 for n in nodes]
        return {
            'edge_index': torch.tensor(edge_index).T,
            'edge_attr': torch.tensor(edge_attrs),
            'x': torch.from_numpy(np.stack(self.embeds[nodes])).to(torch.float)
        }

In [8]:
class CsqaDataset(Dataset):
    def __init__(self, df, tokenizer, graph_miner, concepts):
        super().__init__()
        self.cfg = CFG.data
        self.data = df
        self.tokenizer = tokenizer
        self.graph_miner = graph_miner  # класс составляющий подграф из заданных вершин текста
        self.graph_miner.masking_p = 0.15
        self.concepts = concepts # набор концептов из графа
        
    def __len__(self):
        return len(self.data)

    def prepare_prompt(self, question, concepts, answer):
        s = f"[CSQA] Question: {question} "
        return s, f"Answer: {answer}"
    
    def parse_answer(self, answer, padd_node=160892):
        answer = answer.replace(' ','_')[:]
        if answer in self.concepts:
            res = self.concepts[answer]
            if res == -100:
                return padd_node
            return res
        else:
            return padd_node
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        
        if self.cfg.use_prefix:
            prompts = [
                self.prepare_prompt(row['question'], row['concepts'], i)
                for i in [row['choice_A'],row['choice_B'],row['choice_C'],row['choice_D'],row['choice_E']]
            ]
        else:
            prompts = [
                [row['question'],i] 
                for i in [row['choice_A'],row['choice_B'],row['choice_C'],row['choice_D'],row['choice_E']]
            ]
        
        encodes = [
            self.tokenizer.encode_plus(
                question,answer,
                max_length=self.cfg.max_length,
                truncation='only_first', # обрезаем только сам вопрос, не меняя ответ
                padding='max_length',
                return_tensors='pt'
            )
            for question,answer in prompts
        ]
        answers = [x for x in [row['choice_A'],row['choice_B'],row['choice_C'],row['choice_D'],row['choice_E']]]
        graph_encs = [
            Data(**self.graph_miner.process([self.parse_answer(x) for x in ([a]+row['concepts'])]))
            for a in answers # добавляем вершину потенциального ответа
        ]
        graph_labels = torch.tensor(self.graph_miner.labels)
        self.graph_miner.labels = []
        
        return {
            'labels':torch.cat([torch.stack([x.input_ids.squeeze(0) for x in encodes]),graph_labels],dim=1), # ответы 
            'input_ids':torch.stack([x.input_ids.squeeze(0) for x in encodes]), # последовательность токенов текста
            'attention_mask':torch.stack([x.attention_mask.squeeze(0) for x in encodes]),  # attention mask текста
            'token_type_ids':torch.stack([x.token_type_ids.squeeze(0) for x in encodes]), # token_type_ids текста
            'graph_encodes': graph_encs, # репрезентация графа
        } #

In [9]:
class DreamDataset(Dataset):
    def __init__(self, df, tokenizer, graph_miner, concepts):
        super().__init__()
        self.cfg = CFG.data
        self.data = df
        self.tokenizer = tokenizer
        self.graph_miner = graph_miner # класс составоляющий подграф из заданных вершин текста
        self.graph_miner.masking_p = 0.15
        self.concepts = concepts  # набор концептов из графа
        
    def __len__(self):
        return len(self.data)

    def prepare_prompt(self, dialog, question, concepts, answer):
        s = f"Dialog: {dialog}[SEP] [MRC] Question: {question} "
        return s, f"Answer: {answer}"
    
    def parse_answer(self, answer, padd_node=160892):
        answer = answer.replace(' ','_')[:]
        if answer in self.concepts:
            res = int(self.concepts[answer])
            if res == -100:
                return padd_node
            return res
        else:
            return padd_node

    def parse_dialog(self, dialog):
        return '[SEP]'.join(dialog)
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        dialog = self.parse_dialog(row['dialog'])
        if self.cfg.use_prefix:
            prompts = [
                self.prepare_prompt(dialog, row['question'], row['concepts'], i)
                for i in [row['choice_0'],row['choice_1'],row['choice_2']]
            ]
        else:
            prompts = [
                [row['question'],i] # old ver
                for i in [row['choice_0'],row['choice_1'],row['choice_2']]
            ]
        
        encodes = [
            self.tokenizer.encode_plus(
                question,answer,
                max_length=self.cfg.max_length,
                truncation='only_first',
                padding='max_length',
                return_tensors='pt'
            )
            for question,answer in prompts
        ]
        answers = [x for x in [row['choice_0'],row['choice_1'],row['choice_2']]]
        graph_encs = [
            Data(**self.graph_miner.process([self.parse_answer(x) for x in ([a]+row['concepts'])]))
            for a in answers # добавляем вершину потенциального ответа
        ]
        return {
            'input_ids':torch.stack([x.input_ids.squeeze(0) for x in encodes]), # последовательность токенов текста
            'attention_mask':torch.stack([x.attention_mask.squeeze(0) for x in encodes]),  # attention mask текста
            'token_type_ids':torch.stack([x.token_type_ids.squeeze(0) for x in encodes]), # token_type_ids текста
            'graph_encodes': graph_encs, # репрезентация графа
            'labels': row['labels']
        }

### Специальные DataLoaders

In [10]:
class AlternatingDataLoader:
    """Dataloader, который последовательно выдаёт батчи, сначала все с 1 loader, потом все со 2-го. Нужен для инференса"""
    def __init__(self, loader1, loader2):
        self.loader1 = loader1
        self.loader2 = loader2
        self.iter1 = iter(loader1)
        self.iter2 = iter(loader2)
        self.exhausted1 = False
        self.exhausted2 = False

    def __iter__(self):
        return self

    def get_from_loader1(self):
        try:
            return next(self.iter1)
        except StopIteration:
            self.exhausted1 = True

    def get_from_loader2(self):
        try:
            return next(self.iter2)
        except StopIteration:
            self.exhausted2 = True

    def __next__(self):
        if not self.exhausted1:
            sample1 = self.get_from_loader1()
            if sample1 is not None:
                return sample1
            
        if not self.exhausted2:
            sample2 = self.get_from_loader2()
            if sample2 is not None:
                return sample2
        if self.exhausted1 and self.exhausted2:
            raise StopIteration

    def __len__(self):
        return len(self.loader1) + len(self.loader2)

In [11]:
class RandomDataLoader:
    def __init__(self, loader1, loader2):
        "Семплирует рандомно батчи из 2-х даталоадеров. Нужен для обучения"
        self.loader1 = loader1
        self.loader2 = loader2
        self.iter1 = iter(loader1)
        self.iter2 = iter(loader2)
        self.exhausted1 = False
        self.exhausted2 = False

    def __iter__(self):
        return self

    def get_from_loader1(self):
        try:
            return next(self.iter1)
        except StopIteration:
            self.exhausted1 = True

    def get_from_loader2(self):
        try:
            return next(self.iter2)
        except StopIteration:
            self.exhausted2 = True

    def __next__(self):
        if not self.exhausted1 and not self.exhausted2:
            sample = random.choice([self.get_from_loader1, self.get_from_loader2])()
            if sample is not None:
                return sample
        if not self.exhausted1:
            sample1 = self.get_from_loader1()
            if sample1 is not None:
                return sample1
            
        if not self.exhausted2:
            sample2 = self.get_from_loader2()
            if sample2 is not None:
                return sample2
        if self.exhausted1 and self.exhausted2:
            raise StopIteration

    def __len__(self):
        return len(self.loader1) + len(self.loader2)

In [12]:
def alternate_dataloaders(loader1, loader2):
    iter1, iter2 = iter(loader1), iter(loader2)
    exhausted1, exhausted2 = False, False

    while not (exhausted1 and exhausted2):
        if not exhausted1:
            try:
                yield next(iter1)
            except StopIteration:
                exhausted1 = True

        if not exhausted2:
            try:
                yield next(iter2)
            except StopIteration:
                exhausted2 = True

In [13]:
def sequence_dataloaders(loader1, loader2):
    iter1, iter2 = iter(loader1), iter(loader2)
    exhausted1, exhausted2 = False, False

    while not exhausted1:
        if not exhausted1:
            try:
                yield next(iter1)
            except StopIteration:
                exhausted1 = True
    while not exhausted2:
        if not exhausted2:
            try:
                yield next(iter2)
            except StopIteration:
                exhausted2 = True

### DataModule

In [14]:
class PLDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.data
        self.is_setup = False
        self.is_prepared = False
        
    def prepare_data(self):
        if self.is_prepared: return None

        # загружаем данные csqa
        self.train_df_csqa = make_df_csqa(self.cfg.train_path)
        self.val_df_csqa = make_df_csqa(self.cfg.dev_path)
        self.test_df_csqa = make_df_csqa(self.cfg.test_path,is_test=True)

        # загружаем данные dream
        self.train_df_dream = make_df_dream(self.cfg.dream_train_path, self.cfg.dream_train_concept_path)
        self.val_df_dream = make_df_dream(self.cfg.dream_dev_path, self.cfg.dream_dev_concept_path)
        self.test_df_dream = make_df_dream(self.cfg.dream_test_path, self.cfg.dream_test_concept_path,is_test=True)
        
        # загружаем подобранные вершины для csqa и задаём словари для работы с графом
        self.dop_concepts_csqa = pickle.load(open(self.cfg.dop_concepts_csqa_path, 'rb'))
        self.maper = pickle.load(open(self.cfg.maper_path, 'rb'))
        self.concepts = pickle.load(open(self.cfg.concepts_lst,'rb'))
        self.train_df_csqa['concepts'] = self.dop_concepts_csqa['train_all_cands']
        self.val_df_csqa['concepts'] = self.dop_concepts_csqa['vall_all_cands']
        self.test_df_csqa['concepts'] = self.dop_concepts_csqa['test_all_cands']

        # загружаем подобранные вершины dream
        self.dop_concepts_dream = pickle.load(open(self.cfg.dop_concepts_dream_path, 'rb'))
        self.train_df_dream['concepts'] = self.dop_concepts_dream['train_candidates']
        self.val_df_dream['concepts'] = self.dop_concepts_dream['vall_candidates']
        self.test_df_dream['concepts'] = self.dop_concepts_dream['test_candidates']
        
        # загружаем граф, векторные представления его вершин, класс для предобработки графа, токенизатор
        self.tokenizer = AutoTokenizer.from_pretrained(self.cfg.tokenizer, truncation_side='left') # обрезаем слева, чтобы в первую очередь терять информацию о диалоге, а затем об вопросе
        self.graph_state = pickle.load(open(self.cfg.graph_path, 'rb'))
        self.embeds = np.load(self.cfg.embeds_path)
        self.graph_miner = GraphProcessorV2(self.graph_state['graph'], self.embeds,pad_to_nodes=self.cfg.pad_to_nodes)
        self.is_prepared = True
        
    def setup(self, stage: str):
        if self.is_setup: return None
        # Создаём датасеты для CSQA
        self.train_dataset_csqa = CsqaDataset(self.train_df_csqa,self.tokenizer, self.graph_miner,self.concepts)
        self.val_dataset_csqa = CsqaDataset(self.val_df_csqa,self.tokenizer, self.graph_miner,self.concepts)
        self.predict_dataset_csqa = CsqaDataset(self.test_df_csqa,self.tokenizer, self.graph_miner,self.concepts)

        # Создаём датасеты для Drean
        self.train_dataset_dream = DreamDataset(self.train_df_dream,self.tokenizer, self.graph_miner,self.concepts)
        self.val_dataset_dream = DreamDataset(self.val_df_dream,self.tokenizer, self.graph_miner,self.concepts)
        self.predict_dataset_dream = DreamDataset(self.test_df_dream,self.tokenizer, self.graph_miner,self.concepts)
        self.is_setup = True
    
    def train_dataloader(self):
        csqa_loader = DataLoader(self.train_dataset_csqa,
                         batch_size=self.cfg.batch_size,
                         num_workers=self.cfg.num_workers,
                         collate_fn=collate_fn,
                         pin_memory=True,
                         shuffle=True)
        dream_loader = DataLoader(self.train_dataset_dream,
                         batch_size=self.cfg.batch_size,
                         num_workers=self.cfg.num_workers,
                         collate_fn=collate_fn,
                         pin_memory=True,
                         shuffle=True)
        dataloader_all = RandomDataLoader(csqa_loader, dream_loader)
        return csqa_loader
    
    def val_dataloader(self):
        csqa_loader = DataLoader(self.val_dataset_csqa,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          collate_fn=collate_fn,
                          pin_memory=True,
                          shuffle=False)
        dream_loader = DataLoader(self.val_dataset_dream,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          collate_fn=collate_fn,
                          pin_memory=True,
                          shuffle=False)
        dataloader_all = AlternatingDataLoader(csqa_loader, dream_loader)
        return csqa_loader
    
    def predict_dataloader(self):
        csqa_loader = DataLoader(self.predict_dataset_csqa,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          collate_fn=collate_fn,
                          pin_memory=True,
                          shuffle=False)
        dream_loader = DataLoader(self.predict_dataset_dream,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          collate_fn=collate_fn,
                          pin_memory=True,
                          shuffle=False)
        dataloader_all = AlternatingDataLoader(csqa_loader, dream_loader)
        return csqa_loader


collator = DataCollatorForLanguageModeling(AutoTokenizer.from_pretrained(CFG.data.tokenizer), mlm=True, mlm_probability=0.15) # collator для макскирования токенов

def collate_fn(batches):
    new_batch = {}
    for k in batches[0].keys():
        if k == 'graph_encodes':
            values = []
            for x in batches:
                values += x[k]
            try:
                new_batch[k] = Batch.from_data_list(values)
            except:
                print(values)
                new_batch[k] = Batch.from_data_list(values)
        elif k == 'input_ids':
            new_batch['input_ids'] = torch.stack([x[k] for x in batches])
            batch_size, num_choices = new_batch['input_ids'].size(0), new_batch['input_ids'].size(1)
            input_ids, labels = collator.torch_mask_tokens(new_batch['input_ids'].view(batch_size * num_choices, -1))
            new_batch['input_ids'], new_batch['labels'] = input_ids.view(batch_size, num_choices, -1), labels.view(batch_size, num_choices, -1)
        else:
            new_batch[k] = torch.stack([x[k] for x in batches])
    return new_batch

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Training

In [15]:
class AverageMeter():
    def __init__(self):
        """Класс, для хранения и подсчёта метрик"""
        self.preds = []
        self.labels = []
        self.history = []
    
    def update(self,y_t,y_p):
        self.labels += y_t
        self.preds += y_p
        
    def clean(self):
        self.preds = []
        self.labels = []

    def calc_metrics(self):
        metrics = {}
        metrics['accuracy'] = accuracy_score(self.labels, self.preds)
        
        self.history.append(metrics)
        return metrics

In [16]:
class PLModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.model
        self.model = CustomGraphMLMEncoder(self.cfg)
        self.avg_meter = AverageMeter()
        
    def forward(self, batch):
        output = self.model(**batch)
        return output

    def training_step(self, batch, i):
        out = self(batch)
        loss = out.loss
        self.log('train_loss', loss.item())
        return loss
            
    def validation_step(self, batch, i):
        out = self(batch)
        loss = out.loss
        self.log('val_loss',loss.item())
        preds = out.logits.argmax(dim=-1).view(-1).tolist()
        self.avg_meter.update(batch['labels'].view(-1).tolist(),preds)
    
    def predict_step(self, batch, i):
        out = self(batch)
        logits = out.logits
        return logits.argmax(dim=-1).view(-1).tolist()
                
    def on_validation_epoch_end(self):
        metrics = self.avg_meter.calc_metrics()
        self.log_dict(metrics)
        self.avg_meter.clean()
            
    def configure_optimizers(self):        
        optimizer_parameters = [
            {'params': [p for n, p in self.model.model.named_parameters() if not any(nd in n for nd in self.cfg.no_decay)],
             'lr': self.cfg.lr, 'weight_decay': self.cfg.weight_decay},
            {'params': [p for n, p in self.model.model.named_parameters() if any(nd in n for nd in self.cfg.no_decay)],
             'lr': self.cfg.lr, 'weight_decay': 0.0},
            {'params': [p for n, p in self.model.gnn.named_parameters()],
             'lr': self.cfg.lr_gnn, 'weight_decay': self.cfg.weight_decay},
            {'params': [p for n, p in self.model.fc.named_parameters()],
             'lr': self.cfg.lr_fn, 'weight_decay': self.cfg.weight_decay_fn}
        ]
        
        optim = self.cfg.optim(
            optimizer_parameters,
            lr=self.cfg.lr,
            betas=self.cfg.betas,
            weight_decay=self.cfg.weight_decay,
            eps=self.cfg.eps
        )
        
        if self.cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warmup_steps,
                                                        num_cycles=self.cfg.num_cycles)
        elif self.cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warmup_steps)
        else:
            return optim
        
        scheduler = {'scheduler': scheduler,'interval': 'step', 'frequency': 1}

        return [optim], [scheduler]

In [17]:
dm = PLDataModule()
dm.prepare_data()
dm.setup(0)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Main

In [18]:
def main():
    """Функция запускающая обучение"""
    dm = PLDataModule()
    dm.prepare_data()
    dm.setup(0)

    CFG.model.num_training_steps = len(dm.train_dataloader()) * CFG.model.max_epoches # считаем кол-во warmup шагов
    model = PLModule()

    # запускаем wandb для версионирования
    wandb.login(key="bf4dab65b16e4b3cc78348f00f79cba6db3d5f58")
    os.environ['WANDB_API_KEY'] = "bf4dab65b16e4b3cc78348f00f79cba6db3d5f58"
    wandb.init(project='AIIJC_stage_2',name='deberta-v3-large_inject_graph',config=set_wandb_cfg())

    lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='step')
    checkpoint_cb = pl.callbacks.ModelCheckpoint(
        dirpath='./outputs/', # путь куда будут сохраняться веса
        filename='model_pretrain_{epoch:02d}-{accuracy:.4f}', 
        monitor='accuracy',
        mode='max', # сохраняем  сабмит с максимальным accuracy
        save_last=True # сохраняем последний сабмит
    )

    trainer = pl.Trainer(
        accelerator="gpu",
        precision=32, # тип данных весов модели, в нашем случае fp32
        callbacks = [lr_monitor,checkpoint_cb],
        logger = pl.loggers.WandbLogger(save_code=True),
        log_every_n_steps=1, # логируем на wandb каждый шаг
        accumulate_grad_batches=CFG.model.grad_acum_steps,
        enable_checkpointing=True,
        min_epochs=1,
        devices=1, # кол-во gpu
        check_val_every_n_epoch=1, # валидируемся каждую эпоху
        max_epochs=CFG.model.max_epoches,
        gradient_clip_val=1.0,
        #distributed_backend='ddp'
    )

    trainer.fit(model, datamodule=dm) # запускаем обучение

In [ ]:
main()

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


model.safetensors:   0%|          | 0.00/874M [00:00<?, ?B/s]

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: danasone. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                  | Params | Mode 
--------------------------------------------------------
0 | model | CustomGraphMLMEncoder | 575 M  | train
--------------------------------------------------------

...

Validation: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]